In [2]:
import numpy as np

import data_preprocessing
import neural_metrics

In [3]:
# data_str = 'military_vehicles'
# main_model_name = binary_model_name = additional_model_name = 'vit_b_16'
# secondary_model_name = 'vit_l_16'
# main_lr = binary_lr = 0.0001
# original_num_epochs = secondary_num_epochs = 20
# additional_epoch_num = 30
# binary_num_epochs = 10
# number_of_fine_classes = 24

# data_str = 'imagenet'
# main_model_name = binary_model_name = 'dinov2_vits14'
# secondary_model_name = 'dinov2_vitl14'
# main_lr = binary_lr = 0.000001
# original_num_epochs = 8
# secondary_num_epochs = 2
# binary_num_epochs = 5
# number_of_fine_classes = 42

data_str = 'openimage'
main_model_name = 'tresnet_m'
secondary_model_name = additional_model_name = binary_model_name = 'dinov2_vits14'
main_lr = binary_lr = 0.000001
original_num_epochs = 0
secondary_num_epochs = 2
binary_num_epochs = 4
number_of_fine_classes = 30
additional_epoch_num = 10

preprocessor = data_preprocessing.DataPreprocessor(data_str=data_str)

additional_results_path = 'combined_results'
fine_test_pred = np.load(f'{additional_results_path}/'
                          f"{f'{data_str}_' if data_str != 'military_vehicles' else ''}{main_model_name}_test_fine_pred_BCE_lr{main_lr}_e{original_num_epochs - 1}.npy")
coarse_test_pred = np.load(f'{additional_results_path}/'
                          f"{f'{data_str}_' if data_str != 'military_vehicles' else ''}{main_model_name}_test_coarse_pred_BCE_lr{main_lr}_e{original_num_epochs - 1}.npy")
additional_fine_test_pred = np.load(f'{additional_results_path}/'
                          f'{data_str}_{additional_model_name}_'
                                    f'test_fine_pred_BCE_lr{main_lr}_e{additional_epoch_num - 1}_additional.npy')
additional_coarse_test_pred = np.load(f'{additional_results_path}/'
                          f'{data_str}_{additional_model_name}_'
                                      f'test_coarse_pred_BCE_lr{main_lr}_e{additional_epoch_num - 1}_additional.npy')
fine_test_gt = np.load('data/OpenImage/test_fine/test_true_fine.npy')
coarse_test_gt = np.load('data/OpenImage/test_coarse/test_true_coarse.npy')
fine_acc, fine_f1, _, _ = neural_metrics.get_individual_metrics(pred_data=additional_fine_test_pred,
                                                                true_data=fine_test_gt)
coarse_acc, coarse_f1, _, _ = neural_metrics.get_individual_metrics(pred_data=additional_coarse_test_pred,
                                                          true_data=coarse_test_gt)
print(f'fine acc: {fine_acc}, fine_f1: {fine_f1}\ncoarse acc: {coarse_acc}, coarse_f1: {coarse_f1}')

fine acc: 0.4977503749375104, fine_f1: 0.4764294318151229
coarse acc: 0.8769371771371438, coarse_f1: 0.8627142068930898


In [42]:
error_predictions = (np.where(fine_test_pred != additional_fine_test_pred, 1, 0) | 
                     np.where(fine_test_pred != additional_fine_test_pred , 1, 0))
error_ground_truth = (np.where(fine_test_pred != fine_test_gt , 1, 0) | 
                      np.where( coarse_test_pred != coarse_test_gt, 1, 0))
error_acc, error_f1, _, _ = neural_metrics.get_individual_metrics(pred_data=error_predictions,
                                                          true_data=error_ground_truth)
print(f'error acc: {error_acc}, fine_f1: {error_f1}')


error acc: 0.6824695884019331, fine_f1: 0.6813826668773988
